# Azure ML Training Pipeline for COVID-CXR
This notebook defines an Azure machine learning pipeline for a single training run and submits the pipeline as an experiment to be run on an Azure virtual machine.

In [6]:
# Import statements
import azureml.core
from azureml.core import Experiment
from azureml.core import Workspace, Datastore
from azureml.data.data_reference import DataReference
from azureml.pipeline.core import PipelineData
from azureml.pipeline.core import Pipeline
from azureml.pipeline.steps import PythonScriptStep, EstimatorStep
from azureml.train.dnn import TensorFlow
from azureml.train.estimator import Estimator
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.environment import Environment
from azureml.core.runconfig import RunConfiguration
import shutil

### Register the workspace and configure its Python environment.

In [7]:
# Get reference to the workspace
ws = Workspace.from_config("./ws_config.json")

# Set workspace's environment
env = Environment.from_pip_requirements(name = "covid-cxr_env", file_path = "./../requirements.txt")
env.register(workspace=ws)
runconfig = RunConfiguration(conda_dependencies=env.python.conda_dependencies)
print(env.python.conda_dependencies.serialize_to_string())

# Move AML ignore file to root folder
aml_ignore_path = shutil.copy('./.amlignore', './../.amlignore') 

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - pyyaml==5.2
  - matplotlib==3.1.1
  - tqdm==4.40.2
  - opencv_python==4.1.0.25
  - tensorflow_gpu==2.0.1
  - dill==0.3.0
  - scikit_image==0.15.0
  - numpy==1.17.4
  - pandas==0.25.3
  - pydicom==1.4.2
  - imbalanced_learn==0.6.1
  - tensorboard==2.0.2
  - lime==0.1.1.37
channels:
- anaconda
- conda-forge



### Create references to persistent and intermediate data
Create DataReference objects that point to our raw data on the blob. Configure a PipelineData object to point to preprocessed images stored on the blob.

In [8]:
# Get the blob datastore associated with this workspace
blob_store = Datastore(ws, name='covid_cxr_ds')

# Create data references to folders on the blob
mila_data_dr = DataReference(
    datastore=blob_store,
    data_reference_name="mila_data",
    path_on_datastore="data/covid-chestxray-dataset/")
fig1_data_dr = DataReference(
    datastore=blob_store,
    data_reference_name="fig1_data",
    path_on_datastore="data/Figure1-COVID-chestxray-dataset/")
rsna_data_dr = DataReference(
    datastore=blob_store,
    data_reference_name="rsna_data",
    path_on_datastore="data/rsna/")
training_logs_dr = DataReference(
    datastore=blob_store,
    data_reference_name="training_logs_data",
    path_on_datastore="logs/training/")
models_dr = DataReference(
    datastore=blob_store,
    data_reference_name="models_data",
    path_on_datastore="models/")

# Set up references to pipeline data (intermediate pipeline storage).
processed_pd = PipelineData(
    "processed_data",
    datastore=blob_store,
    output_name="processed_data",
    output_mode="mount")

### Compute Target
Specify and configure the compute target for this workspace. If a compute cluster by the name we specified does not exist, create a new compute cluster.

In [9]:
CT_NAME = "nd12s-clust"         # Name of our compute cluster
VM_SIZE = "STANDARD_ND12S"      # Specify the Azure VM for execution of our pipeline

# Set up the compute target for this experiment
try:
    compute_target = AmlCompute(ws, CT_NAME)
    print("Found existing compute target.")
except ComputeTargetException:
    print("Creating new compute target")
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = VM_SIZE,
                                                                max_nodes = 4)    
    compute_target = ComputeTarget.create(ws, CT_NAME, provisioning_config)  # Create the compute cluster
    
    # Wait for cluster to be provisioned
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20) 
    
print("Azure Machine Learning Compute attached")
print("Compute targets: ", ws.compute_targets)
compute_target = ws.compute_targets[CT_NAME]

Found existing compute target.
Azure Machine Learning Compute attached
Compute targets:  {'nd12s-clust': AmlCompute(workspace=Workspace.create(name='covid-cxr-ws', subscription_id='4d8a1c27-3a23-4f81-8fec-5b598044652e', resource_group='covid-cxr-rg'), name=nd12s-clust, id=/subscriptions/4d8a1c27-3a23-4f81-8fec-5b598044652e/resourceGroups/covid-cxr-rg/providers/Microsoft.MachineLearningServices/workspaces/covid-cxr-ws/computes/nd12s-clust, type=AmlCompute, provisioning_state=Failed, location=eastus, tags=None)}


### Define pipeline and submit experiment.
Define the steps of an Azure machine learning pipeline. Create an Azure Experiment that will run our pipeline. Submit the experiment to the execution environment.

In [10]:
# Define preprocessing step the ML pipeline
step1 = PythonScriptStep(name="preprocess_step",
                         script_name="azure/preprocess_step/preprocess_step.py",
                         arguments=["--miladatadir", mila_data_dr, "--fig1datadir", fig1_data_dr, 
                                    "--rsnadatadir", rsna_data_dr, "--preprocesseddir", processed_pd],
                         inputs=[covid_data_dr, rsna_data_dr],
                         outputs=[processed_pd],
                         compute_target=compute_target, 
                         source_directory="./../",
                         runconfig=runconfig,
                         allow_reuse=True)

# Define training step in the ML pipeline
est = TensorFlow(source_directory='./../',
                   script_params=None,
                   compute_target=compute_target,
                   entry_script='azure/train_step/train_step.py',
                   pip_packages=['tensorboard', 'pandas', 'dill', 'numpy', 'imblearn', 'matplotlib', 'scikit-image', 'matplotlib',
                                'pydicom', 'opencv-python', 'tqdm', 'scikit-learn'],
                   use_gpu=True,
                   framework_version='2.0')
step2 = EstimatorStep(name="estimator_train_step", 
                      estimator=est, 
                      estimator_entry_script_arguments=["--preprocesseddir", processed_pd, "--traininglogsdir", training_logs_dr,
                                                       "--modelsdir", models_dr],
                      runconfig_pipeline_params=None, 
                      inputs=[processed_pd, training_logs_dr, models_dr], 
                      outputs=[], 
                      compute_target=compute_target)

# Construct the ML pipeline from the steps
steps = [step1, step2]
single_train_pipeline = Pipeline(workspace=ws, steps=steps)
single_train_pipeline.validate()

# Define a new experiment and submit a new pipeline run to the compute target.
experiment = Experiment(workspace=ws, name='SingleTrainExperiment_v2')
experiment.submit(single_train_pipeline, regenerate_outputs=False)
print("Pipeline is submitted for execution")

# Move AML ignore file back to original folder
aml_ignore_path = shutil.move(aml_ignore_path, './.amlignore') 

Step preprocess_step is ready to be created [0e61c112]
Step estimator_train_step is ready to be created [d437ce8a]
Data reference raw_covid_data is ready to be created [a5ad5ad0], (Consumers of this data will generate new runs.)
Data reference raw_rsna_data is ready to be created [ed650778], (Consumers of this data will generate new runs.)
Data reference training_logs_data is ready to be created [9cb5ac9a], (Consumers of this data will generate new runs.)
Data reference models_data is ready to be created [0049f6d7], (Consumers of this data will generate new runs.)
Created step preprocess_step [0e61c112][b445f9d7-dd35-4888-9915-090ffe119620], (This step will run and generate new outputs)
Created step estimator_train_step [d437ce8a][202f9132-b74b-4775-9a0f-2e724b60b922], (This step will run and generate new outputs)


KeyboardInterrupt: 